In [2]:
import sqlite3
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from utils.utils import get_citation_doi
from utils.utils import get_paper_citation_pairs
from acm_api.acm import acm_meta
from evaluation.evaluate import Train_Test_Split

<h2>Processing the input file with doi of papers</h2>

In [3]:
doi_list = []
data = {}

# reading the input file with doi of papers
with open('doi/doi.txt', 'r') as doi_file:
    lines = doi_file.readlines()
    
for line in lines:
    doi_list.append(line.strip())
    
con = sqlite3.connect('papers.db')
cur = con.cursor()

is_table_exists = cur.execute(
    """
    SELECT name FROM sqlite_master WHERE type='table'
    AND name='PAPERS'; """).fetchall()

if is_table_exists == []:
    cur.execute(""" 
        CREATE TABLE PAPERS (doi TEXT, data TEXT);
    """)
    
print('processing doi....')
for doi in tqdm(doi_list):
    cursor = con.execute("SELECT * FROM PAPERS WHERE doi=?;", (doi,))
    fetch_data = cursor.fetchone()
    
    if fetch_data is None:
        temp = acm_meta(doi)
        insert_statement = 'INSERT INTO PAPERS(doi, data) VALUES(?, ?)'
        
        con.execute(insert_statement, (doi, str(temp)))
        data[doi] = get_citation_doi(temp)
        
    else:
        data[doi] = get_citation_doi(fetch_data[1])

con.commit()
con.close()

processing doi....


  0%|          | 0/8 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/249 [00:00<?, ?it/s]

extracting citation's doi....


...

  0%|          | 0/146 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/18 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/136 [00:00<?, ?it/s]

In [4]:
# Paper's doi and it's citations
data

{'10.1145/3463475': ['10.5555/2969033.2969125',
  '10.1145/3301282',
  '10.5555/2987189.2987190',
  '10.5555/3327546.3327685',
  '10.5555/3454287.3455618',
  '10.5555/3295222.3295378',
  '10.5555/3305381.3305578',
  '10.5555/2969239.2969405',
  '10.5555/3157096.3157340',
  '10.5555/3305381.3305404',
  '10.5555/3295222.3295327',
  '10.1007/s11263-015-0816-y',
  '10.1007/s11042-017-5457-4',
  '10.1007/s11227-017-2218-0',
  '10.1145/3343031.3350944',
  '10.1109/34.3909',
  '10.1145/882262.882269',
  '10.1145/3072959.3073659',
  '10.1109/TIFS.2019.2907973',
  '10.5555/3305890.3305954',
  '10.5555/3294771.3294810',
  '10.5555/3326943.3326948',
  '10.1109/TIP.2019.2916751',
  '10.5555/3157096.3157106',
  '10.1145/3240508.3240618',
  '10.1145/3335203.3335724',
  '10.5555/3305381.3305430',
  '10.5555/3305381.3305573',
  '10.5555/3294771.3294838',
  '10.1145/311535.311556',
  '10.5555/3454287.3454527',
  '10.1109/TIFS.2019.2891116',
  '10.5555/3294996.3295075',
  '10.5555/3298483.3298649',
  '1

<h2>Code for creating the rating matrix</h2>

In [5]:
citation_set = set()
for key in data.keys():
    citation_set.update(data[key])

index_ = data.keys()
df = pd.DataFrame(np.zeros((len(index_), len(citation_set))), index=index_, columns=citation_set)

for i in index_:
    for j in citation_set:
        if j in data[i]:
            df.loc[i][j] = 1

df.to_csv('citation-web/matrix-way/citation-matrix.csv')

<h2>Creating the citation pair</h2>

In [6]:
citation_pair_path = get_paper_citation_pairs(df, 'citation-pairs.txt')

writing citation pairs....


  0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
########################################
	# Experiment 1
	# Plain CF
	
	
	########################################


	########################################
	# Experiment 1
	# Plain CF
	
	# TO DO
	########################################